In [ ]:
!pip install essentia essentia-tensorflow
!cp -r /content/drive/MyDrive/music .
!cp -r /content/drive/MyDrive/models ./models
%load_ext autoreload
%autoreload 2

In [3]:
# @title Download file

import os
import requests
from tqdm import tqdm

model_url = "https://essentia.upf.edu/models/feature-extractors/discogs-effnet/discogs_multi_embeddings-effnet-bs64-1.json" # @param {type:"string"}
model_name = os.path.basename(model_url)
dirpath = "models" # @param {type:"string"}
# Create the directory if it doesn't exist
if not os.path.exists(dirpath):
    os.makedirs(dirpath)

# Download the model with progress
response = requests.get(model_url, stream=True)
total_size = int(response.headers.get('content-length', 0))
block_size = 1024
progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

# Save the model to disk
with open(os.path.join(dirpath, model_name), "wb") as f:
    for chunk in response.iter_content(block_size):
        progress_bar.update(len(chunk))
        f.write(chunk)

progress_bar.close()

print(f"Downloaded model to: {os.path.join('models', model_name)}")

100%|██████████| 2.22k/2.22k [00:00<00:00, 3.62MiB/s]

Downloaded model to: models/discogs_multi_embeddings-effnet-bs64-1.json


In [ ]:
import os
import numpy as np
from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs
# tracks = [f"./music/{k}" for k in os.listdir("music")]
# tracks

In [5]:
modeldir = "models"
modelname = "discogs_track_embeddings-effnet-bs64-1.pb"
model = TensorflowPredictEffnetDiscogs(graphFilename=f"{modeldir}/{modelname}", output="PartitionedCall:1")

In [13]:
audio1 = MonoLoader(filename=tracks[-1], sampleRate=16000, resampleQuality=4)()
audio2 = MonoLoader(filename=tracks[-2], sampleRate=16000, resampleQuality=4)()

In [14]:
# %timeit
embedding1 = model(audio1)
embedding2 = model(audio2)

In [ ]:
def bilenear_product(emb1, emb2):
    projection_positive = np.matmul(BL_PW, np.transpose(emb2))
    compatibily = np.matmul(emb1, projection_positive)
    return compatibily

In [23]:
dur = min(embedding1.shape[0], embedding2.shape[0])
np.linalg.norm(embedding1[:dur,:] - embedding2[:dur,:])


270.95322

In [ ]:
def distance(emb1, emb2):
    return np.linalg.norm(emb1 - emb2)

# get embedding of everu track
embeddings = [model(MonoLoader(filename=track, sampleRate=16000, resampleQuality=4)()) for track in tracks]
# make a distance matrix
distances = np.zeros((len(embeddings), len(embeddings)))
for i in range(len(embeddings)):
    for j in range(len(embeddings)):
        distances[i,j] = distance(embeddings[i], embeddings[j])

import matplotlib.pyplot as plt
# distance is a scalar, plot it as a heatmap
plt.imshow(distances)
plt.colorbar()



In [ ]:
for row in distances:
    # print(row)
    # get the index of the minimum value other than 0
    min_index = np.argmin(row[np.nonzero(row)])
    print(min_index, row[min_index])